In [2]:
using BenchmarkTools, MacroTools, Lazy, Statistics, SpecialFunctions

In [2]:
using Printf # This is not needed as it is defined in my startup file => /Users/malcolm/.julia/config/startup.jl

---
# Multiple Dispatch
---

In [3]:
recip(x::Number) = (x == zero(typeof(x))) ?  error("Invalid reciprocal") : one(typeof(x))/x

recip (generic function with 1 method)

In [4]:
recip(2)

0.5

In [5]:
recip(11//17)

17//11

In [6]:
recip(11 + 17im)

0.02682926829268293 - 0.041463414634146344im

In [7]:
aa = rand(3)
recip(aa)

LoadError: MethodError: no method matching recip(::Vector{Float64})

[0mClosest candidates are:
[0m  recip([91m::Number[39m)
[0m[90m   @[39m [35mMain[39m [90m[4mIn[3]:1[24m[39m


In [8]:
recip(a::Array) = map(recip,a)
recip(aa)

3-element Vector{Float64}:
 1.1928657450413518
 3.2229386614132087
 5.681446315503077

In [9]:
map(sin,recip(aa))

3-element Vector{Float64}:
  0.929430236117361
 -0.08125632406691903
 -0.5660768706711086

In [10]:
bb = [2.1 3.2 4.3; 9.8 8.7 7.6]
recip(bb)

2×3 Matrix{Float64}:
 0.47619   0.3125    0.232558
 0.102041  0.114943  0.131579

In [11]:
cc = recip(aa)'.*recip(bb)

2×3 Matrix{Float64}:
 0.568031  1.00717   1.32127
 0.121721  0.370453  0.747559

## Code Generation

In [12]:
incr(x) = x + 1
incr(2)

3

In [13]:
@code_native incr(2)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	_julia_incr_1713                ## -- Begin function julia_incr_1713
	.p2align	4, 0x90
_julia_incr_1713:                       ## @julia_incr_1713
; ┌ @ In[12]:1 within `incr`
	.cfi_startproc
## %bb.0:                               ## %top
; │┌ @ int.jl:87 within `+`
	leaq	1(%rdi), %rax
; │└
	retq
	.cfi_endproc
; └
                                        ## -- End function
.subsections_via_symbols


In [14]:
@code_native incr(2.7)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.section	__TEXT,__literal8,8byte_literals
	.p2align	3                               ## -- Begin function julia_incr_1736
LCPI0_0:
	.quad	0x3ff0000000000000              ## double 1
	.section	__TEXT,__text,regular,pure_instructions
	.globl	_julia_incr_1736
	.p2align	4, 0x90
_julia_incr_1736:                       ## @julia_incr_1736
; ┌ @ In[12]:1 within `incr`
	.cfi_startproc
## %bb.0:                               ## %top
	movabsq	$LCPI0_0, %rax
; │┌ @ promotion.jl:410 within `+` @ float.jl:408
	vaddsd	(%rax), %xmm0, %xmm0
; │└
	retq
	.cfi_endproc
; └
                                        ## -- End function
.subsections_via_symbols


In [18]:
@code_native incr(2//7)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	_julia_incr_1747                ## -- Begin function julia_incr_1747
	.p2align	4, 0x90
_julia_incr_1747:                       ## @julia_incr_1747
; ┌ @ In[12]:1 within `incr`
	.cfi_startproc
## %bb.0:                               ## %top
	subq	$8, %rsp
	.cfi_def_cfa_offset 16
	movq	%rsi, %rax
; │┌ @ rational.jl:328 within `+`
; ││┌ @ checked.jl:174 within `checked_add`
; │││┌ @ checked.jl:141 within `add_with_overflow`
	movq	(%rsi), %rsi
; ││└└
; ││┌ @ checked.jl:296 within `checked_mul`
; │││┌ @ checked.jl:243 within `mul_with_overflow`
	movq	8(%rax), %rdx
; ││└└
; ││┌ @ checked.jl:174 within `checked_add`
; │││┌ @ checked.jl:141 within `add_with_overflow`
	movq	%rsi, %rax
	addq	%rdx, %rax
; │││└
; │││ @ checked.jl:175 within `checked_add`
	jo	LBB0_2
## %bb.1:                               ## %L17
; │└└
	movq	%rax, (%rdi)
	movq	%rdx, 8(%rdi)
	movq	%rdi, %rax
	popq	%rcx
	retq
LBB0_2:               

In [19]:
@code_native incr(2.0 + 7.0im)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.section	__TEXT,__literal8,8byte_literals
	.p2align	3                               ## -- Begin function julia_incr_1754
LCPI0_0:
	.quad	0x3ff0000000000000              ## double 1
	.section	__TEXT,__text,regular,pure_instructions
	.globl	_julia_incr_1754
	.p2align	4, 0x90
_julia_incr_1754:                       ## @julia_incr_1754
; ┌ @ In[12]:1 within `incr`
	.cfi_startproc
## %bb.0:                               ## %top
	movq	%rdi, %rax
; │┌ @ complex.jl:329 within `+` @ promotion.jl:410 @ float.jl:408
	vmovsd	(%rsi), %xmm0                   ## xmm0 = mem[0],zero
; ││ @ complex.jl:329 within `+`
; ││┌ @ complex.jl:14 within `Complex` @ complex.jl:14
	vmovsd	8(%rsi), %xmm1                  ## xmm1 = mem[0],zero
	movabsq	$LCPI0_0, %rcx
; ││└
; ││ @ complex.jl:329 within `+` @ promotion.jl:410 @ float.jl:408
	vaddsd	(%rcx), %xmm0, %xmm0
; │└
	vmovsd	%xmm0, (%rdi)
	vmovsd	%xmm1, 8(%rdi)
	retq
	.cfi_endproc
;

In [20]:
dump(:(incr(2.7)))

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol incr
    2: Float64 2.7


In [21]:
@code_lowered incr(2.7)

CodeInfo(
1 ─ %1 = x + 1
└──      return %1
)

In [22]:
@code_typed incr(2.7)

CodeInfo(
1 ─ %1 = Base.add_float(x, 1.0)::Float64
└──      return %1
) => Float64

In [23]:
@code_llvm(incr(2.7))

;  @ In[12]:1 within `incr`
define double @julia_incr_2351(double %0) #0 {
top:
; ┌ @ promotion.jl:410 within `+` @ float.jl:408
   %1 = fadd double %0, 1.000000e+00
; └
  ret double %1
}


---
# Metaprogramming
---

In [24]:
ex1 = :((x^2 + y^2 - 2*x*y)^0.5)

:(((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5)

In [25]:
ex2 = quote
(x^2 + y^2 - 2*x*y)^0.5
end

quote
    #= In[25]:2 =#
    ((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5
end

In [27]:
x = 1.1
y = 2.5
eval(ex1)

1.4

In [28]:
@eval ex1

:(((x ^ 2 + y ^ 2) - 2 * x * y) ^ 0.5)

In [33]:
@eval $ex1

1.4

In [34]:
@eval $(ex1) === eval(ex1)

true

In [35]:
ex1 == ex2

false

In [36]:
eval(ex1) == eval(ex2)

true

In [38]:
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol ^
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol -
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol x
                3: Int64 2
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol y
                3: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((4,))
            1: Symbol *
            2: Int64 2
            3: Symbol x
            4: Symbol y
    3: Float64 0.5


In [39]:
Meta.show_sexpr(ex1)

(:call, :^, (:call, :-, (:call, :+, (:call, :^, :x, 2), (:call, :^, :y, 2)), (:call, :*, 2, :x, :y)), 0.5)

In [40]:
function traverse!(ex, symbols) end

traverse! (generic function with 1 method)

In [41]:
# If ex is a symbol push it onto the tree
function traverse!(ex::Symbol, symbols)
  push!(symbols, ex)
end

traverse! (generic function with 2 methods)

In [42]:
# Main processing function.
# We need to distinguish between a :call and
# other arguments (recursively).
#
function traverse!(ex::Expr, symbols)
  if ex.head == :call # function call
    for arg in ex.args[2:end]
      traverse!(arg, symbols) # recursive
    end
  else
    for arg in ex.args
      traverse!(arg, symbols) # recursive
    end
  end
end

traverse! (generic function with 3 methods)

In [44]:
# Define a wrapper function around traverse! function(s)
# Define an empty symbols array and push any found.
# Notice the use of unique to prune the symbols array
#
function traverse(ex::Expr)
  symbols = Symbol[]
  traverse!(ex, symbols)
  return unique(symbols) # Don't output duplicates
end

traverse (generic function with 1 method)

In [155]:
traverse(:(mad(a,b,c)))

3-element Vector{Symbol}:
 :a
 :b
 :c

In [46]:
ex3 = :(2 * 3 + 7)
eval(ex3)

13

In [47]:
dump(ex3)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol *
        2: Int64 2
        3: Int64 3
    3: Int64 7


In [48]:
dump(ex3.args[2])

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Int64 2
    3: Int64 3


In [49]:
ex3.args[1] = :*
(ex3.args[2]).args[1] = :+
eval(ex3)

35

In [50]:
ex4 = :((2+3) * 7)
dump(ex4)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol *
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 2
        3: Int64 3
    3: Int64 7


---
# Macros
---

In [51]:
macro pout(ex)
  if typeof(ex) == Expr
    println(ex.args)
  end
  return ex
end

@pout (macro with 1 method)

In [52]:
x = 1.1; @pout x

1.1

In [53]:
y = 2.3
@pout (x^2 + y^2 - 2*x*y)^0.5

Any[:^, :((x ^ 2 + y ^ 2) - 2 * x * y), 0.5]


1.1999999999999997

In [54]:
macro dotimes(n, body)
  quote
    for i = 1:$(esc(n))
      $(esc(body))
    end
  end
end

@dotimes (macro with 1 method)

In [55]:
@dotimes 3 print("Hi")

HiHiHi

In [56]:
i = 0
@dotimes 3 [global i += 1; println(i*i)]

1
4
9


In [57]:
macro until(condition, block)
  quote
    while true
      $(esc(block))
      if $(esc(condition))
        break
      end
    end
  end
end

@until (macro with 1 method)

In [58]:
i = 0
@until (i >= 3) [global i += 1; println(i*i)]

1
4
9


In [59]:
macro iif(cond, body1, body2)
  :(if !$cond
    $(esc(body1))
  else
    $(esc(body2))
  end)
end


@iif (macro with 1 method)

In [62]:
using SpecialFunctions

In [64]:
n = 10
@iif (n < 1) factorial(n) ArgumentError("$n not positive")

3628800

In [65]:
n = -1
@iif (n < 1) factorial(n) ArgumentError("$n not positive")

ArgumentError("-1 not positive")

In [66]:
function kempner(n::Integer)
  @assert n > 0
  s = 0.0
  r9 = r"9" # Match a string containing a 9
  for i in 1:n
    if (match(r9,string(i)) == nothing)
      s += 1.0/float(i)
    end
  end
  return s
end

kempner (generic function with 1 method)

In [67]:
[kempner(10^i) for i in 1:7]

7-element Vector{Float64}:
  2.8178571428571426
  4.78184876508206
  6.590720190283038
  8.223184402866208
  9.692877792106202
 11.015651849872553
 12.206153722565858

In [68]:
macro bmk(fex, n::Integer)
  quote
    let s = 0.0
      if $(esc(n)) > 0
        val = $(esc(fex))
        for i = 1:$(esc(n))
          local t0 = Base.time_ns()
          local val = $(esc(fex))
          s += Base.time_ns() - t0
        end
        return s/($(esc(n)) * 10e9)
     else
       end
    end
  end
end

@bmk (macro with 1 method)

In [69]:
@bmk kempner(10^7) 10

0.26684991671

In [70]:
import Base.@time
time(n::Integer) = kempner(n)

time (generic function with 1 method)

In [71]:
@time time(10^7)

  2.621327 seconds (35.65 M allocations: 1.675 GiB, 8.25% gc time)


12.206153722565858

In [72]:
macroexpand(Main,:(@assert n > 0))

:(if n > 0
      nothing
  else
      Base.throw(Base.AssertionError("n > 0"))
  end)

In [73]:
@macroexpand @assert n > 0

:(if n > 0
      nothing
  else
      Base.throw(Base.AssertionError("n > 0"))
  end)

In [74]:
n = -1; @assert n > 0

LoadError: AssertionError: n > 0

In [75]:
@macroexpand @dotimes 3 [global i += 1; println(i*i)]

quote
    #= In[54]:3 =#
    for var"#55#i" = 1:3
        #= In[54]:4 =#
        [global i += 1; println(i * i)]
        #= In[54]:5 =#
    end
end

In [76]:
Base.remove_linenums!(@macroexpand @bmk kempner(10^7) 10)

quote
    let var"#56#s" = 0.0
        if 10 > 0
            var"#59#val" = kempner(10 ^ 7)
            for var"#57#i" = 1:10
                local var"#58#t0" = (Main.Base).time_ns()
                local var"#59#val" = kempner(10 ^ 7)
                var"#56#s" += (Main.Base).time_ns() - var"#58#t0"
            end
            return var"#56#s" / (10 * 1.0e10)
        else
        end
    end
end

In [78]:
function poly_native(x, a...)
  p=zero(x)
  for i = 1:length(a)
    p = p + a[i] * x^(i-1)
  end
  return p
end

poly_native (generic function with 1 method)

In [79]:
f_native(x) = poly_native(x,1,2,3,4,5)

f_native (generic function with 1 method)

In [80]:
f_native(2.1)

152.71450000000004

In [81]:
function poly_horner(x, a...)
  b = zero(x)
  for i = length(a):-1:1
    b = a[i] + b * x
  end
  return b
end

poly_horner (generic function with 1 method)

In [84]:
f_horner(x) = poly_horner(x,1,2,3,4,5)
round(f_horner(2.1),digits=4)

152.7145

In [85]:
mad(x,a,b) = a*x + b
mad(2.1,5,4)

14.5

In [86]:
macro horner(x, p...)
  ex = esc(p[end])
  for i = length(p)-1:-1:1
    ex = :(mad(t, $ex, $(esc(p[i]))))
  end
  Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

In [87]:
round(@horner(2.1,1,2,3,4,5), digits=4)

152.7145

In [88]:
@macroexpand @horner 2.1 1 2 3 4 5

quote
    var"#68#t" = 2.1
    Main.mad(var"#68#t", Main.mad(var"#68#t", Main.mad(var"#68#t", Main.mad(var"#68#t", 5, 4), 3), 2), 1)
end

## Macrotools

In [89]:
using MacroTools

In [90]:
ex = :(1 + (2 + 3) + 4)

:(1 + (2 + 3) + 4)

In [92]:
p = postwalk(ex) do x
  x isa Integer ? factorial(x) : x
end

:(1 + (2 + 6) + 24)

In [93]:
eval(p)

33

In [95]:
map(x -> @show(x), [1,2,3,4]);

x = 1
x = 2
x = 3
x = 4


In [96]:
postwalk(ex) do x
  @show x
end

x = :+
x = 1
x = :+
x = 2
x = 3
x = :(2 + 3)
x = 4
x = :(1 + (2 + 3) + 4)


:(1 + (2 + 3) + 4)

In [98]:
@capture(ex, a_ + b_ + c_)

true

In [99]:
b

:(2 + 3)

In [100]:
a*eval(b) + c 

9

In [101]:
reduce(+, 1:10)

55

In [102]:
plus(a, b) = :($a + $b)

plus (generic function with 1 method)

In [103]:
p = reduce(plus, 1:10)

:(((((((((1 + 2) + 3) + 4) + 5) + 6) + 7) + 8) + 9) + 10)

In [104]:
eval(p)

55

In [105]:
k = 2
pp = [:($((-1)^k)*x^$(1+2k) / $(factorial(1+2k))) for k = 0:5]

6-element Vector{Expr}:
 :((1 * x ^ 1) / 1)
 :((-1 * x ^ 3) / 6)
 :((1 * x ^ 5) / 120)
 :((-1 * x ^ 7) / 5040)
 :((1 * x ^ 9) / 362880)
 :((-1 * x ^ 11) / 39916800)

In [106]:
reduce(plus,pp)

:((((((1 * x ^ 1) / 1 + (-1 * x ^ 3) / 6) + (1 * x ^ 5) / 120) + (-1 * x ^ 7) / 5040) + (1 * x ^ 9) / 362880) + (-1 * x ^ 11) / 39916800)

In [107]:
x = 2.1
eval(reduce(plus,pp))

0.8632069372306019

### Lazy

In [133]:
using Lazy
import Lazy: cycle, range, drop, take

In [109]:
fibs = @lazy 0:1:(fibs + drop(1, fibs));

In [110]:
take(15, fibs)

List: (0 1 1 2 3 5 8 13 21 34 55 …)

In [111]:
@> π/6 sin exp

1.648721270700128

In [120]:
f(x,y) = (x + y)^2

f (generic function with 1 method)

In [121]:
@> π/6 f(1.6)

4.509671759722594

In [122]:
esquares = @>> range() map(x -> x^2) filter(iseven)

List: (4 16 36 64 100 144 196 256 324 400 484 …)

In [124]:
take(7, esquares)

List: (4 16 36 64 100 144 196)

In [130]:
isprime(n) =
  @>> primes begin
  takewhile(x -> x<=sqrt(n))
  map(x -> n % x == 0)
  any;!
end

isprime (generic function with 1 method)

In [131]:
primes = filter(isprime, range(2))

List: (2 3 5 7 11 13 17 19 23 29 31 …)

In [132]:
isprime(113)

true

### Generated functions

In [141]:
@generated function mad(a,b,c)
  Core.println("Calculating a*b + c")
  return :(a * b + c)
end

mad (generic function with 1 method)

In [142]:
mad(2.3,1.7,1.1)
Calculating: a*b + c
5.01

Calculating a*b + c


LoadError: UndefVarError: `Calculating` not defined

In [144]:
mad(2.3,1.7,2.1)

6.01

In [145]:
mad(2.3,1.7,1)

Calculating a*b + c


4.91

In [146]:
function pdims(x::Array{T,N}) where {T,N}
  s = 1
  for i = 1:N
    s = s * size(x, i)
  end
  return s
end

pdims (generic function with 1 method)

In [147]:
@generated function gpdims(x::Array{T,N}) where {T,N}
  ex = :(1)
  for i = 1:N
    ex = :(size(x, $i) * $ex)
  end
  return ex
end

gpdims (generic function with 1 method)

In [148]:
aa = [rand() for i = 1:1000]
aax = reshape(aa,10,5,5,4)
size(aax)

(10, 5, 5, 4)

In [149]:
pdims(aax) == gpdims(aax)

true

In [150]:
@code_lowered pdims(aax)

CodeInfo(
1 ─       s = 1
│   %2  = 1:$(Expr(:static_parameter, 2))
│         @_3 = Base.iterate(%2)
│   %4  = @_3 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_3
│         i = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│   %10 = s
│   %11 = Main.size(x, i)
│         s = %10 * %11
│         @_3 = Base.iterate(%2, %9)
│   %14 = @_3 === nothing
│   %15 = Base.not_int(%14)
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return s
)

In [151]:
@code_lowered gpdims(aax)

CodeInfo(
    @ In[147]:1 within `gpdims`
   ┌ @ In[147] within `macro expansion`
1 ─│ %1 = Main.size(x, 4)
│  │ %2 = Main.size(x, 3)
│  │ %3 = Main.size(x, 2)
│  │ %4 = Main.size(x, 1)
│  │ %5 = %4 * 1
│  │ %6 = %3 * %5
│  │ %7 = %2 * %6
│  │ %8 = %1 * %7
└──│      return %8
   └
)

In [152]:
@code_native pdims(aax)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	_julia_pdims_4256               ## -- Begin function julia_pdims_4256
	.p2align	4, 0x90
_julia_pdims_4256:                      ## @julia_pdims_4256
; ┌ @ In[146]:1 within `pdims`
	.cfi_startproc
## %bb.0:                               ## %arraysize.3
; │ @ In[146]:4 within `pdims`
; │┌ @ array.jl:148 within `size`
	movq	32(%rdi), %rax
; │└
; │┌ @ int.jl:88 within `*`
	imulq	24(%rdi), %rax
	imulq	40(%rdi), %rax
	imulq	48(%rdi), %rax
; │└
; │ @ In[146]:6 within `pdims`
	retq
	.cfi_endproc
; └
                                        ## -- End function
	.section	__TEXT,__cstring,cstring_literals
	.p2align	4                               ## @_j_str1
L__j_str1:
	.asciz	"arraysize: dimension out of range"

.subsections_via_symbols


In [153]:
@code_native gpdims(aax)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 11, 0
	.globl	_julia_gpdims_4258              ## -- Begin function julia_gpdims_4258
	.p2align	4, 0x90
_julia_gpdims_4258:                     ## @julia_gpdims_4258
; ┌ @ In[147]:1 within `gpdims`
	.cfi_startproc
## %bb.0:                               ## %top
; │┌ @ In[147] within `macro expansion`
; ││┌ @ array.jl:148 within `size`
	movq	40(%rdi), %rax
; ││└
; ││┌ @ int.jl:88 within `*`
	imulq	48(%rdi), %rax
	imulq	32(%rdi), %rax
	imulq	24(%rdi), %rax
; ││└
	retq
	.cfi_endproc
; └└
                                        ## -- End function
.subsections_via_symbols


---

In [5]:
module ModInts
export ModInt
import Base: +, -, *, /, inv
struct ModInt{n} <: Integer
  k::Int
  ModInt{n}(k) where n = new(mod(k,n))
end
Base.show(io::IO, k::ModInt{n}) where n =
print(io, get(io, :compact, false) ? k.k : "$(k.k) mod $n")
(a::ModInt{n} + b::ModInt{n}) where n = ModInt{n}(a.k+b.k)
(a::ModInt{n} - b::ModInt{n}) where n = ModInt{n}(a.k-b.k)
(a::ModInt{n} * b::ModInt{n}) where n = ModInt{n}(a.k*b.k)
-(a::ModInt{n}) where n = ModInt{n}(-a.k)
inv(a::ModInt{n}) where n = ModInt{n}(invmod(a.k, n))
(a::ModInt{n} / b::ModInt{n}) where n = a*inv(b)
Base.promote_rule(::Type{ModInt{n}}, ::Type{Int}) where n = ModInt{n}
Base.convert(::Type{ModInt{n}}, i::Int) where n = ModInt{n}(i)
end

Main.ModInts

In [6]:
using Main.ModInts
m1 = ModInt{11}(2)
m2 = ModInt{11}(7)
m3 = 3*m1 + m2

2 mod 11

In [7]:
mm = reshape([ModInt{11}(rand(0:10)) for i = 1:100],10,10)
ma = [ModInt{11}(rand(0:10)) for i = 1:10]
mm.*ma'

10×10 Matrix{ModInt{11}}:
 0  10  0   9   6  0   0  5  8  0
 0   4  0   2  10  0  10  7  0  0
 0  10  0   6   2  0  10  2  9  0
 0   4  0   2   6  0   6  7  8  0
 0  10  0   8   5  0   2  6  7  0
 0   1  0  10   9  0   9  6  7  0
 0   3  0   2   4  0   6  4  2  0
 0   7  0   1   1  0   8  8  1  0
 0   3  0   2   1  0   6  8  2  0
 0   7  0   1   1  0   1  6  8  0

In [8]:
using Test

In [9]:
x = 1
@test x == 1

Test Passed

In [10]:
@test x == 2

Test Failed at In[10]:1
  Expression: x == 2
   Evaluated: 1 == 2



LoadError: [91mThere was an error during testing[39m

In [11]:
a = rand(10)

10-element Vector{Float64}:
 0.6732681321910536
 0.5114958830723525
 0.647550435513172
 0.30762745755034604
 0.3670177036820047
 0.5854018270986431
 0.16050204402577206
 0.6557146600948265
 0.46730693904326503
 0.10359924299610068

In [12]:
@test_throws BoundsError a[11] = 0.1

Test Passed
      Thrown: BoundsError

In [13]:
@test_throws DomainError a[11] = 0.1

Test Failed at In[13]:1
  Expression: a[11] = 0.1
    Expected: DomainError
      Thrown: BoundsError



LoadError: [91mThere was an error during testing[39m

In [14]:
@testset "Trigonometric identities" begin
x = 2/3*π
@test sin(-x) ≈ -sin(x)
@test cos(-x) ≈ -cos(x)
@test sin(2x) ≈ 2*sin(x)*cos(x)
@test cos(2x) ≈ cos(x)^2 - sin(x)^2
end;

Trigonometric identities: Test Failed at In[14]:4
  Expression: cos(-x) ≈ -(cos(x))
   Evaluated: -0.4999999999999998 ≈ 0.4999999999999998

Stacktrace:
 [1] macro expansion
   @ /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Test/src/Test.jl:478 [inlined]
 [2] macro expansion
   @ In[14]:4 [inlined]
 [3] macro expansion
   @ /Applications/Julia-1.9.app/Contents/Resources/julia/share/julia/stdlib/v1.9/Test/src/Test.jl:1498 [inlined]
 [4] top-level scope
   @ In[14]:2
Test Summary:            | Pass  Fail  Total  Time
Trigonometric identities |    3     1      4  1.8s


LoadError: [91mSome tests did not pass: 3 passed, 1 failed, 0 errored, 0 broken.[39m

---
## Order Pairs
---

In [39]:
module OrdPairs

import Base: +,-,*,/,==,!=,>,<,>=,<=
import Base: abs,conj,inv,zero,one,show
import LinearAlgebra: transpose,adjoint,norm

struct OrdPair{T<:Number}
    a::T
    b::T
end

OrdPair(x::Number) = OrdPair(x, zero(Number))

value(u::OrdPair)   = u.a
epsilon(u::OrdPair) = u.b

zero(::Type{OrdPairs.OrdPair}) = OrdPair(zero(Number),zero(Number))
one(::Type{OrdPairs.OrdPair})  = OrdPair(one(Number),zero(Number))

abs(u::OrdPair)  = abs(value(u))
norm(u::OrdPair) = norm(value(u))

+(u::OrdPair, v::OrdPair) = OrdPair(value(u)+value(v), epsilon(u)+epsilon(v))
-(u::OrdPair, v::OrdPair) = OrdPair(value(u)-value(v), epsilon(u)-epsilon(v))
*(u::OrdPair, v::OrdPair) = OrdPair(value(u)*value(v), epsilon(u)*value(v)+value(u)*epsilon(v))
/(u::OrdPair, v::OrdPair) = OrdPair(value(u)/value(v),(epsilon(u)*value(v)-value(u)*epsilon(v))/(value(v)*value(v)))

==(u::OrdPair, v::OrdPair) = norm(u) == norm(v)
!=(u::OrdPair, v::OrdPair) = norm(u) != norm(v)
>(u::OrdPair, v::OrdPair)  = norm(u) > norm(v)
>=(u::OrdPair, v::OrdPair) = norm(u) >= norm(v)
<(u::OrdPair, v::OrdPair)  = norm(u) < norm(v)
<=(u::OrdPair, v::OrdPair) = norm(u) <= norm(v)

function show(io::IO,u::OrdPair)
 op::String = (epsilon(u) < 0.0) ? " - " : " + ";
 print(io,value(u),op,abs(epsilon(u))," ϵ")
end

export OrdPair

end

Main.OrdPairs

In [40]:
using Main.OrdPairs

In [41]:
p1 = OrdPair(2.3,-1.7)

LoadError: MethodError: no method matching OrdPair(::Float64, ::Float64)

[0mClosest candidates are:
[0m  OrdPair(::Number)
[0m[90m   @[39m [36mMain.OrdPairs[39m [90m[4mIn[23]:10[24m[39m


In [32]:
p2/p1

LoadError: UndefVarError: `p2` not defined